In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('Set2')

#한글폰트 가져오기
from matplotlib import rc
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False #음수에 (-)표현

# Warning 메세지를 뜨지 않게 해줌
import warnings
warnings.filterwarnings('ignore')

from pyproj import Proj, transform
from scipy.spatial import KDTree
from haversine import haversine

In [27]:
df_14=pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0525/df_14_16.csv",encoding='cp949')
df_17=pd.read_csv("C:/Users/wjdtj/OneDrive - dgu.ac.kr/데이터 분석/교통사고/0525/df_17_19.csv",encoding='cp949')

In [28]:
# 나이 
df_14=df_14[(df_14['환자연령']<130)&(df_14['환자연령']>0)]
df_17=df_17[(df_17['환자연령']<130)&(df_17['환자연령']>0)]


In [29]:
# 위치정보
df_14.dropna(subset=['위치정보X','위치정보Y'],inplace=True)
df_17.dropna(subset=['위치정보X','위치정보Y'],inplace=True)

In [33]:
from pyproj import Proj, transform

proj_5181 = Proj(init='epsg:5181')
proj_4326 = Proj(init='epsg:4326')

def convert_5181_to_4326(x, y):
    # EPSG:5181에서 EPSG:4326으로 변환
    lon, lat = transform(proj_5181, proj_4326, x, y)
    return lat, lon


In [34]:
df_14['위도'],df_14['경도']=convert_5181_to_4326(df_14['위치정보X'], df_14['위치정보Y'])
df_17['위도'],df_17['경도']=convert_5181_to_4326(df_17['위치정보X'], df_17['위치정보Y'])

In [35]:
df_14 = df_14[df_14['경도']>126.85]
df_17 = df_17[df_17['경도']>126.85]

## 밀도

In [30]:
# 밀도 계산
def calculate_density(df, x_col='위치정보X', y_col='위치정보Y', radius=100): #radius (float): 거리 반경 (기본값 100 미터).

    # UTM 좌표에서 경위도 좌표로 변환
    utm_proj = Proj(init = 'epsg:5181')
    latlon_proj = Proj(init='epsg:4326')
    
    df = df.dropna(subset=[x_col, y_col]) #na를 데이터 프레임에서 아예 없애버림
    lon, lat = transform(utm_proj, latlon_proj, df[x_col].values, df[y_col].values)
    df['경도'] = lon
    df['위도'] = lat
    
    # KDTree를 이용한 거리 계산 및 밀도 추정
    coords = np.vstack([lon, lat]).T
    tree = KDTree(coords)
    
    # 각 점 주변 반경 내 점의 개수를 센다
    densities = tree.query_ball_point(coords, r=radius / 111000)  # 111000 미터는 1도 위도
    density = [len(points) for points in densities]
    
    return density

In [31]:
df_14['밀도'] = calculate_density(df_14)
df_17['밀도'] = calculate_density(df_17)

In [15]:
# UTM좌표를 WGS84로 변환
proj_UTM = Proj(init = 'epsg:5181')
proj_WGS84 = Proj(init='epsg:4326')
df_14['경도'],df_14['위도'] = transform(proj_UTM,proj_WGS84,df_14['위치정보X'],df_14['위치정보Y'])


from sklearn.neighbors import KernelDensity

data = df_14[['위도','경도']]

# 커널 밀도 추정을 위한 100미터에 해당하는 밴드폭 설정
bandwidth = 0.0009

kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
kde.fit(data)

# 각 데이터 포인트의 로그 밀도 값 계산
log_density_values = kde.score_samples(data)

# 로그 밀도 값을 지수 함수로 변환하여 실제 밀도 값 계산
density_values = np.exp(log_density_values)

df_14['밀도_kde'] = density_values

In [23]:
# UTM좌표를 WGS84로 변환
proj_UTM = Proj(init = 'epsg:5181')
proj_WGS84 = Proj(init='epsg:4326')
df_17['경도'],df_17['위도'] = transform(proj_UTM,proj_WGS84,df_17['위치정보X'],df_17['위치정보Y'])


from sklearn.neighbors import KernelDensity

data = df_17[['위도','경도']]

# 커널 밀도 추정을 위한 100미터에 해당하는 밴드폭 설정
bandwidth = 0.0009

kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
kde.fit(data)

# 각 데이터 포인트의 로그 밀도 값 계산
log_density_values = kde.score_samples(data)

# 로그 밀도 값을 지수 함수로 변환하여 실제 밀도 값 계산
density_values = np.exp(log_density_values)

df_17['밀도_kde'] = density_values

In [ ]:
# 학교

In [ ]:
# 교차로

In [ ]:
# 버스정류장